In [2]:
#coding=utf-8
import requests
from bs4 import Tag
from bs4 import BeautifulSoup
 
 
def getHtml(url):
    page = requests.get(url)
    html =page.text
    return html
 
def getText(html):
    get_text = Tag.get_text
    soup = BeautifulSoup(html, 'html.parser')
    
    author_info = soup.find_all('div', class_='atl-info')
    listauthor  = [x.get_text() for x in author_info]
        
    list_info = soup.find_all('div', class_='bbs-content')
    listtext  = [x.get_text() for x in list_info]
 
    global i
    if i > 1:
        listtext = [""] + listtext
    
    for x in range(len(listauthor)):
        if "楼主" in listauthor[x]:
            print (listtext[x].strip())
            
if __name__=='__main__':
    for i in range(1,6):
        url  = ("http://bbs.tianya.cn/post-feeling-4286798-%s.shtml" % str(i))
        html = getHtml(url)
        getText(html)

In [47]:
import re
import requests
import xlsxwriter
from bs4 import BeautifulSoup

def get_html(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"
    }
    try:
        response = requests.get(url, headers = headers)
        response.encoding = 'gbk'  # 编码方式问题
        html = response.text
        return html
    except:
        return ""


def getMovie(html):
    allmovies = []
    movieurl = []
    ftpurl = []
    soup = BeautifulSoup(html, "html.parser")
    url_info = soup.find_all("a", class_="ulink")
    for info in url_info:
        pattern = "《.*?》"
        movies = re.findall(pattern, info.get_text())
        for movie in movies:
            allmovies.append(movie)
        reurl = "https://www.dydytt.net" + info.get("href")
        movieurl.append(reurl)
    for url in movieurl:
        try:
            html = getHtml(url)
            reg = r'href="(ftp:.+?)">'
            imgre = re.compile(reg)
            imaglist, *_ = re.findall(imgre, html)
            print(imaglist)
            ftpurl.append(imaglist)
        except:
            print("")
            ftpurl.append("")
    return allmovies, ftpurl

if __name__ == "__main__":
    workbook = xlsxwriter.Workbook("movie_source.xlsx")
    worksheet = workbook.add_worksheet()
    row = 0
    for i in range(1, 166):
        print("Visiting the " + str(i) + " page")
        url = "https://www.dydytt.net/html/tv/oumeitv/list_9_" + str(i) + ".html"
        html = get_html(url)
        if not html:
            print("Access Exception")
            continue
        movie, ftp = getMovie(html)
        for item in zip(movie, ftp):
            worksheet.write(row, 0. item[0])
            worksheet.write(row, 4, item[1])
            row = row + 1
    workbook.close()
    print("Download finished")
        
        

Visiting the 1 page



In [5]:
# Ajax demo
# The key point of ajax is find the request url and the changing parameters.
# leanred from https://cuiqingcai.com/202253.html
import requests
import logging
import pymongo

logging.basicConfig(level=logging.INFO, 
                    format="%(asctime)s - %(levelname)s: %(message)s")

INDEX_URL = "https://spa1.scrape.center/api/movie/?limit={limit}&offset={offset}"
DETAIL_URL = "https://spa1.scrape.center/api/movie/{id}"

MONGO_CONNECTION_STRING = "mongodb://localhost:27017"
MONGO_DB_NAME = "movies"  # mongodb 数据库名称
MONGO_COLLECTION_NAME = "movies" # mongodb 的集合名称

LIMIT = 10
TOTAL_PAGE = 10

def scrape_api(url):
    logging.info("scraping %s...", url)
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        logging.error("get invalid status code %s while scraping %s", response.status_code, url)
    except requests.RequestException:
        logging.error("error occurred while scraping %s", url, exc_info=True)

def scrape_index(page):
    url = INDEX_URL.format(limit=LIMIT, offset=LIMIT * (page -1))
    return scrape_api(url)

def scrape_detail(id):
    url = DETAIL_URL.format(id=id)
    return scrape_api(url)

client = pymongo.MongoClient(MONGO_CONNECTION_STRING)
db = client["movies"]
collection = db["movies"]

def save_data(data):
    collection.update_one({
        "name": data.get("name")
    }, {
        "$set": data
    }, upsert=True)

def main():
    for page in range(1, TOTAL_PAGE + 1):
        index_data = scrape_index(page)
        for item in index_data.get("results"):
            id = item.get("id")
            detail_data = scrape_detail(id)
            logging.info("detail data %s", detail_data)
            save_data(detail_data)
            logging.info("data saved successfully")

if __name__ == "__main__":
    main()

In [3]:
import requests 
import logging 
import time

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s : %(message)s")

TOTAL_NUMBER = 100
URL = "https://httpbin.org/delay/5"

start_time = time.time()

for _ in range(1, TOTAL_NUMBER + 1):
    logging.info("scraping %s", URL)
    response = requests.get(URL)
end_time = time.time()

logging.info("total cost time %s seconds", end_time - start_time)

In [7]:
import asyncio

async def execute(x):
    print("Number: ", x)

coroutine = execute(1)
print("Coroutine: ", coroutine)
print("After calling execute")

loop = asyncio.get_event_loop()
loop.run_until_complete(coroutine)
print("After calling loop")

In [ ]:
import asyncio
import aiohttp
import requests
import time

start = time.time()

async def get(url):
    session = aiohttp.ClientSession()
    response = await session.get(url)
    await response.text()
    await session.close()
    return response

async def request():
    url = "https://httpbin.org/delay/5"
    print("waiting for", url)
    response = await get(url)
    print(f"Get response from {url}, response: {response}")

tasks = [asyncio.ensure_future(request()) for _ in range(10)]
loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(tasks))

end = time.time()
print("Cost time: ", end - start)

In [10]:
import asyncio
import aiohttp
import time

start = time.time()

async def get(url):
    session = aiohttp.ClientSession()
    response = await session.get(url)
    await response.text()
    await session.close()
    return response

async def request():
    url = 'https://httpbin.org/delay/5'
    print('Waiting for', url)
    response = await get(url)
    print('Get response from', url, 'response', response)

tasks = [asyncio.ensure_future(request()) for _ in range(10)]
loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(tasks))

end = time.time()
print('Cost time:', end - start)

In [ ]:
import asyncio
import aiohttp
import time


def test(number):
    start = time.time()

    async def get(url):
        session = aiohttp.ClientSession()
        response = await session.get(url)
        await response.text()
        await session.close()
        return response

    async def request():
        url = 'https://www.baidu.com/'
        await get(url)

    tasks = [asyncio.ensure_future(request()) for _ in range(number)]
    loop = asyncio.get_event_loop()
    loop.run_until_complete(asyncio.wait(tasks))

    end = time.time()
    print('Number:', number, 'Cost time:', end - start)

for number in [1, 3, 5, 10, 15, 30, 50, 75, 100, 200, 500]:
    test(number)

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.support.wait import WebDriverWait 

browser = webdriver.Chrome()
try:
    browser.get("https://www.baidu.com")
    input = browser.find_element_by_id("kw")
    input.send_keys("python")
    input.send_keys(Keys.ENTER)
    # 显示等待
    wait = WebDriverWait(browser, 10)
    wait.until(EC.presence_of_element_located((By.ID, "content-left")))
    print(browser.current_url)
    print(browser.get_cookies())
    # page_source 网页源代码
    print(browser.page_source)
finally:
    browser.close()

TimeoutException: Message: 


In [ ]:
from selenium import webdriver
from selenium.webdriver import ChromeOptions

option = ChromeOptions()
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_experimental_option('useAutomationExtension', False)
browser = webdriver.Chrome(options=option)
browser.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
    'source': 'Object.defineProperty(navigator, "webdriver", {get: () => undefined})'
})
browser.get('https://antispider1.scrape.center/')

In [ ]:
from selenium import webdriver
from selenium.webdriver import ChromeOptions

# option = ChromeOptions()
# option.add_experimental_option('excludeSwitches', ['enable-automation'])
# option.add_experimental_option('useAutomationExtension', False)
# browser = webdriver.Chrome(options=option)
browser = webdriver.Chrome()
# browser.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
#     'source': 'Object.defineProperty(navigator, "webdriver", {get: () => undefined})'
# })
browser.get('https://antispider1.scrape.center/')

In [15]:
import requests
from urllib.parse import urljoin

BASE_URL = 'https://login2.scrape.center/'
LOGIN_URL = urljoin(BASE_URL, '/login')
INDEX_URL = urljoin(BASE_URL, '/page/1')
USERNAME = 'admin'
PASSWORD = 'admin'

# requests直接调用post、get等方法，每次请求都是一个独立的请求，这都相当于是
# 新开一个浏览器打开这些链接，所以这两次请求对应的session并不是同一个，这里我们模拟第一个session登录，并不能影响第二个session的状态
# 因此模拟登录也就无效了。
response_login = requests.post(LOGIN_URL, data={
    'username': USERNAME,
    'password': PASSWORD
}, allow_redirects=False) # requests可以自动处理重定向，我们在模拟登录的时候要加上allow_redirects参数将其设置为False


cookies = response_login.cookies
print("Cookies: ", cookies)

response_index = requests.get(INDEX_URL, cookies=cookies)
print('Response Status', response_index.status_code)
print('Response URL', response_index.url)

Cookies:  <RequestsCookieJar[<Cookie sessionid=7c310hdo1i80258o84e7t30p50qjohos for login2.scrape.center/>]>
Response Status 200
Response URL https://login2.scrape.center/page/1


In [26]:
import requests
from urllib.parse import urljoin

BASE_URL = 'https://login2.scrape.center/'
LOGIN_URL = urljoin(BASE_URL, '/login')
INDEX_URL = urljoin(BASE_URL, '/page/1')
USERNAME = 'admin'
PASSWORD = 'admin'

# 无须关心Cookie的处理和传递问题，声明一个Session对象
# 然后每次调用请求的时候都直接使用Session对象的post或get方法
session = requests.Session()

response_login = session.post(LOGIN_URL, data={
    "username": USERNAME,
    "password": PASSWORD
})

cookies = session.cookies
print("Cookies: ", cookies)

response_index = session.get(INDEX_URL)
print("Response status: ", response_index.status_code)
print("Response URL: ", response_index.url)

Cookies:  <RequestsCookieJar[<Cookie sessionid=a7jcxnw3y3exsw038339txjaj0vr9ree for login2.scrape.center/>]>
Response status:  200
Response URL:  https://login2.scrape.center/page/1


In [29]:
# 使用selenium模拟登录，获取cookies

from urllib.parse import urljoin
from selenium import webdriver
import requests
import time

BASE_URL = 'https://login2.scrape.center/'
LOGIN_URL = urljoin(BASE_URL, '/login')
INDEX_URL = urljoin(BASE_URL, '/page/1')
USERNAME = 'admin'
PASSWORD = 'admin'

browser = webdriver.Chrome()
browser.get(BASE_URL)
browser.find_element_by_css_selector('input[name="username"]').send_keys(USERNAME)
browser.find_element_by_css_selector('input[name="password"]').send_keys(PASSWORD)
browser.find_element_by_css_selector('input[type="submit"]').click()
time.sleep(10)

cookies = browser.get_cookies()
print("Cookies: ", cookies)
browser.close()

session = requests.Session()
for cookie in cookies:
    session.cookies.set(cookie["name"], cookie["value"])
    
response_index = session.get(INDEX_URL)
print("Response Status: ", response_index.status_code)
print("Response URL", response_index.url)

Cookies:  [{'domain': 'login2.scrape.center', 'expiry': 1677740342, 'httpOnly': True, 'name': 'sessionid', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'rqy0w4gz0ujmpibrw5d70ehhmrnu606b'}]
Response Status:  200
Response URL https://login2.scrape.center/page/1


In [31]:
import requests
import time

start = time.time()
session = requests.Session()
for _ in range(10):
    session.get("http://httpbin.org/delay/1")
    print(f"Finished { _ + 1} requests")
end = time.time()
print("Cost time: ", end - start)

Finished 1 requests
Finished 2 requests
Finished 3 requests
Finished 4 requests
Finished 5 requests
Finished 6 requests
Finished 7 requests
Finished 8 requests
Finished 9 requests
Finished 10 requests
Cost time:  14.763051509857178


In [32]:
import requests_cache
import time

start = time.time()
session = requests_cache.CachedSession("demo_cache")

for _ in range(10):
    session.get("http://httpbin.org/delay/1")
    print(f"Finished { _ + 1} requests")
end = time.time()
print("Cost time: ", end - start)

c:\Users\star\AppData\Local\Programs\Python\Python36\lib\site-packages\pymongo\pyopenssl_context.py:26: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography.x509 import load_der_x509_certificate as _load_der_x509_certificate


Finished 1 requests
Finished 2 requests
Finished 3 requests
Finished 4 requests
Finished 5 requests
Finished 6 requests
Finished 7 requests
Finished 8 requests
Finished 9 requests
Finished 10 requests
Cost time:  1.8296372890472412


In [41]:
import requests
import requests_cache
import time

# use_temp 缓存文件夹便会使用系统的临时目录，而不会在代码区创建缓存文件夹
requests_cache.install_cache("demo_cache", backend="filesystem", use_temp=True)

# use_cache_dir参数，缓存文件夹便会使用系统的专用缓存文件夹，而不会在代码区创建缓存文件夹。
# requests_cache.install_cache("demo_cache", backend="filesystem", use_cache_dir=True)

# 使用 redis 
# backend = requests_cache.RedisCache(host="localhost", port=6379)
# requests_cache.install_cache("demo_cache", backend=backend)

start = time.time()
session = requests.Session()
for _ in range(10):
    session.get("http://httpbin.org/delay/1")
    print(f"Finished {_ + 1} requests")
end = time.time()
print("Cost time: ", end - start)

Finished 1 requests
Finished 2 requests
Finished 3 requests
Finished 4 requests
Finished 5 requests
Finished 6 requests
Finished 7 requests
Finished 8 requests
Finished 9 requests
Finished 10 requests
Cost time:  0.027999401092529297


In [47]:
import time
import requests
import requests_cache

# allowable_codes(200, ) 只有响应为200的会缓存
requests_cache.install_cache("demo_cache2", allowable_methods=["POST"])


# 匹配 url，针对哪种 pattern 的url缓存多久
# site_1.com 的内容会缓存 30s， site_2.com/static 的内容永远不会过期
# urls_expire_after = {"*.site_1.com": 30, "site_2.com/static": -1}
# request_cache.install_cache(
#     "demo_cache2", urls_expire_after=urls_expire_after
# )

start = time.time()
session = requests.Session()
for i in range(10):
    session.get("http://httpbin.org/delay/1")
    print(f"Finished {i + 1} requests")
end = time.time()
print("Cost time for get", end - start)
start = time.time()

for i in range(10):
    session.post("http://httpbin.org/delay/1")
    print(f"Finished {i + 1} requests")
end = time.time()
print("Cost time for post", end - start)

Finished 1 requests
Finished 2 requests
Finished 3 requests
Finished 4 requests
Finished 5 requests
Finished 6 requests
Finished 7 requests
Finished 8 requests
Finished 9 requests
Finished 10 requests
Cost time for get 14.257228374481201
Finished 1 requests
Finished 2 requests
Finished 3 requests
Finished 4 requests
Finished 5 requests
Finished 6 requests
Finished 7 requests
Finished 8 requests
Finished 9 requests
Finished 10 requests
Cost time for post 0.017030000686645508


In [1]:
import time
import re
import tesserocr
from selenium import webdriver
from io import BytesIO
from PIL import Image
from retrying import retry
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import numpy as np


def preprocess(image):
    image = image.convert('L')
    array = np.array(image)
    array = np.where(array > 50, 255, 0)
    image = Image.fromarray(array.astype('uint8'))
    return image


@retry(stop_max_attempt_number=10, retry_on_result=lambda x: x is False)
def login():
    browser.get('https://captcha7.scrape.center/')
    browser.find_element_by_css_selector('.username input[type="text"]').send_keys('admin')
    browser.find_element_by_css_selector('.password input[type="password"]').send_keys('admin')
    captcha = browser.find_element_by_css_selector('#captcha')
    image = Image.open(BytesIO(captcha.screenshot_as_png))
    image = preprocess(image)
    captcha = tesserocr.image_to_text(image)
    captcha = re.sub('[^A-Za-z0-9]', '', captcha)
    browser.find_element_by_css_selector('.captcha input[type="text"]').send_keys(captcha)
    browser.find_element_by_css_selector('.login').click()
    try:
        WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//h2[contains(., "登录成功")]')))
        time.sleep(10)
        browser.close()
        return True
    except TimeoutException:
        return False


if __name__ == '__main__':
    browser = webdriver.Chrome()
    login()

In [16]:
from selenium import webdriver
from selenium.webdriver import ChromeOptions

option = ChromeOptions()
option.add_argument('--headless')
browser = webdriver.Chrome(options=option)
browser.set_window_size(1366, 768)
browser.get('https://www.baidu.com')
browser.get_screenshot_as_file('preview.png')

True

In [8]:
import cv2

img = cv2.imread(r"C:\Users\star\Desktop\a.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

_, threshold = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
opening = cv2.morphologyEx(threshold, cv2.MORPH_OPEN, kernel, iterations=2)

contours, _ = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for cnt in contours:
    area = cv2.contourArea(cnt)
    if area > 100:
        x, y, w, h = cv2.boundingRect(cnt)
        img[y:y+h, x:x+w] = (255, 255, 255)
        
cv2.imwrite(r"C:\Users\star\Desktop\b.jpg", img)

True

In [9]:
import cv2

img = cv2.imread(r"C:\Users\star\Desktop\a.jpg")
img = cv2.resize(img, None, fx=0.2, fy=0.2, interpolation=cv2.INTER_CUBIC)

img_blur = cv2.GaussianBlur(img, (25, 25), 0)

img_pixelate = cv2.resize(img_blur, None, fx=0.05, fy=0.05, interpolation=cv2.INTER_NEAREST)
img_pixelate = cv2.resize(img_pixelate, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)

img_mosaic = cv2.resize(img_pixelate, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)
img_restored = cv2.addWeighted(img, 0.8, img_mosaic, 0.2, 0)

cv2.imwrite(r"C:\Users\star\Desktop\b.jpg", img_restored)

True

In [2]:
import time
for item in "Yu".split():
    print('\n'.join([''.join([(item[(x-y) % len(item)] if ((x*0.05)**2+(y*0.1)**2-1)**3-(x*0.05)**2*(y*0.1)**3 <= 0 else ' ') for x in range(-30, 30)]) for y in range(12, -12, -1)]))
    time.sleep(1.5);

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
import time
from loguru import logger

COUNT = 1000

for i in range(1, COUNT + 1):
    try:
        browser = webdriver.Chrome()
        wait = WebDriverWait(browser, 10)
        browser.get('https://captcha1.scrape.center/')
        button = wait.until(
            EC.element_to_be_clickable(
                (By.CSS_SELECTOR, '.el-button')
            )
        )
        button.click()
        captcha = wait.until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, '.geetest_slicebg.geetest_absolute')
            )
        )
        time.sleep(5)
        captcha.screenshot(f'data/captcha/images/captcha_{i}.png')
    except WebDriverException as e:
        logger.error(f'webdriver error occurred {e.msg}')
    finally:
        browser.close()

2023-02-20 09:44:08.383 | ERROR    | __main__:<module>:30 - webdriver error occurred 


KeyboardInterrupt: 

In [ ]:
export https_proxy=http://127.0.0.1:33210 http_proxy=http://127.0.0.1:33210 all_proxy=socks5://127.0.0.1:33211

In [14]:
# urllib set proxy
# http/https

from urllib.error import URLError
from urllib.request import ProxyHandler, build_opener


# 使用 ProxyHander 设置代理，参数是字典类型，键名为协议类型，键值是代理
# 当请求为 http 协议的时候，会使用 http 键名对应的代理，当请求连接为 https 协议的时候，会使用 https 键名对应的代理
proxy = "127.0.0.1:33210"
proxy_handler = ProxyHandler({
    "http": "http://" + proxy,
    "https": "https://" + proxy
})
# 创建完 ProxyHander 对象之后，需要利用 build_opener 方法传入该对象来创建一个Opener
# 这样就相当于此 Opener 已经设置好代理了，接下来直接调用 Opener 对象的 open 方法，即可访问我们所想要的链接
opener = build_opener(proxy_handler)

try:
    response = opener.open("https://httpbin.org/get")
    print(response.read().decode("utf-8"))
except URLError as e:
    print(e.reason)

{
  "args": {}, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Host": "httpbin.org", 
    "User-Agent": "Python-urllib/3.6", 
    "X-Amzn-Trace-Id": "Root=1-63f2d570-600b36e754bbf93f64b3b6c8"
  }, 
  "origin": "103.135.102.146", 
  "url": "https://httpbin.org/get"
}



In [17]:
# socks5 代理

import socks
import socket
from urllib import request
from urllib.error import URLError

socks.set_default_proxy(socks.SOCKS5, '127.0.0.1', 33211)
socket.socket = socks.socksocket
try:
    response = request.urlopen('https://httpbin.org/get')
    print(response.read().decode('utf-8'))
except URLError as e:
    print(e.reason)

{
  "args": {}, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Host": "httpbin.org", 
    "User-Agent": "Python-urllib/3.6", 
    "X-Amzn-Trace-Id": "Root=1-63f2d6ef-32ffc96229b50a384c902e4e"
  }, 
  "origin": "103.135.102.146", 
  "url": "https://httpbin.org/get"
}



In [24]:
# requests set proxy

import requests

# If account password authentication is required, "username:password@127.0.0.1:33210"
proxy = "127.0.0.1:33210"
proxies = {
    "http": "http://" + proxy,
    "https": "http://" + proxy
}
try:
    response = requests.get("https://httpbin.org/get", proxies=proxies)
    print(response.text)
except requests.exceptions.ConnectionError as e:
    print("Error", e.args)

{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.27.1", 
    "X-Amzn-Trace-Id": "Root=1-63f2d817-6b0db1d919d2d59f35c0f620"
  }, 
  "origin": "103.135.102.146", 
  "url": "https://httpbin.org/get"
}



In [25]:
# requests socks5

import requests

proxy = "127.0.0.1:33211"
proxies = {
    "http": "socks5://" + proxy,
    "https": "socks5://" + proxy
}

try:
    response = requests.get("https://httpbin.org/get", proxies=proxies)
    print(response.text)
except requests.exceptions.ConnectionError as e:
    print("Error: ", e.args)

{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.27.1", 
    "X-Amzn-Trace-Id": "Root=1-63f2d938-0a1caf585bf4fe1d347d9409"
  }, 
  "origin": "103.135.102.146", 
  "url": "https://httpbin.org/get"
}



In [ ]:
import requests
import socks
import socket

socks.set_default_proxy(socks.SOCKSS, "127.0.0.1", "33211")
socket.socket = socks.socksocket
try:
    response = requests.get("https://httpbin.org/get")
    print(response.text)
except request.exceptions.ConnectionError as e:
    print("Error")

In [26]:
# httpx proxy

import httpx

proxy = "127.0.0.1:33210"
proxies = {
    "http://": "http://" + proxy,
    "https://": "http://" + proxy
}

with httpx.Client(proxies=proxies) as client:
    response = client.get("http://httpbin.org/get")
    print(response.text)

{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-httpx/0.22.0", 
    "X-Amzn-Trace-Id": "Root=1-63f2e06a-32d5956e72fb156a2b80ca0d"
  }, 
  "origin": "103.135.102.146", 
  "url": "http://httpbin.org/get"
}



In [28]:
import httpx
from httpx_socks import SyncProxyTransport

trasport = SyncProxyTransport.from_url(
    "socks5://127.0.0.1:33211"
)

with httpx.Client(transport=trasport) as client:
    response = client.get("http://httpbin.org/get")
    print(response.text)

{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-httpx/0.22.0", 
    "X-Amzn-Trace-Id": "Root=1-63f2e126-34574a8b029a14513455cc2b"
  }, 
  "origin": "103.135.102.146", 
  "url": "http://httpbin.org/get"
}



In [33]:
# selenium proxy

from selenium import webdriver

proxy = "127.0.0.1:33211"
options = webdriver.ChromeOptions()
options.add_argument('--proxy-server=http://' + proxy)
browser = webdriver.Chrome(options=options)
browser.get('https://httpbin.org/get')
print(browser.page_source)
browser.close()

In [1]:
# aiohttp proxy

import asyncio
import aiohttp

proxy = "http://127.0.0.1:33211"

async def main():
    async with aiohttp.ClientSession() as session:
        async with session.get("https://httpbin.org/get", proxy=proxy) as response:
            print(await response.text())

if __name__ == "__main__":
    asyncio.get_event_loop().run_until_complete(main())

In [ ]:
# aiohttp socks

import aiohttp
import asyncio
from aiohttp_socks import ProxyConnector

connector = ProxyConnector.from_url("socks5://127.0.0.1:33211")

async def main():
    async with aiohttp.ClientSession(connector=connector) as session:
        async with session.get("https://httpbin.org/get") as response:
            print(await response.text())
            
if __name__ == "__main__":
    asyncio.get_event_loop().run_until_complete(main())